In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from scipy.special import boxcox1p
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, Imputer
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

c:\users\administrator.xhgtfpw25hudl36\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [22]:
#导入数据及预览前几行
data = pd.read_csv(open("E:/practice/机器学习/互金/dataSet/LoanStats3d.csv", 'rb'), low_memory=False, skiprows=[0])
data = data.drop(data.index[-3:], axis=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421094 entries, 0 to 421093
Columns: 144 entries, id to settlement_term
dtypes: float64(107), object(37)
memory usage: 465.8+ MB


In [23]:
#统计每列的缺失值情况
check_null = data.isnull().sum(axis=0).sort_values(ascending=False)/float(len(data)) #查看缺失值比例
print(check_null[check_null > 0.2]) # 查看缺失比例大于20%的属性。

id                                            1.000000
sec_app_mths_since_last_major_derog           1.000000
sec_app_earliest_cr_line                      1.000000
sec_app_inq_last_6mths                        1.000000
sec_app_mort_acc                              1.000000
sec_app_open_acc                              1.000000
sec_app_revol_util                            1.000000
sec_app_open_act_il                           1.000000
sec_app_num_rev_accts                         1.000000
sec_app_chargeoff_within_12_mths              1.000000
sec_app_collections_12_mths_ex_med            1.000000
revol_bal_joint                               1.000000
member_id                                     1.000000
url                                           1.000000
desc                                          0.999893
dti_joint                                     0.998791
annual_inc_joint                              0.998786
verification_status_joint                     0.998786
orig_proje

In [5]:
# 删除缺失值超过40%的列
thresh_count = len(data) * 0.4 # 设定阀值
data = data.dropna(thresh=thresh_count, axis=1 ) #若某一列数据缺失的数量超过阀值就会被删除
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421094 entries, 0 to 421093
Data columns (total 87 columns):
loan_amnt                     421094 non-null float64
funded_amnt                   421094 non-null float64
funded_amnt_inv               421094 non-null float64
term                          421094 non-null object
int_rate                      421094 non-null object
installment                   421094 non-null float64
grade                         421094 non-null object
sub_grade                     421094 non-null object
emp_title                     397220 non-null object
emp_length                    397277 non-null object
home_ownership                421094 non-null object
annual_inc                    421094 non-null float64
verification_status           421094 non-null object
issue_d                       421094 non-null object
loan_status                   421094 non-null object
pymnt_plan                    421094 non-null object
purpose                       421094

ValueError: Cannot describe a DataFrame without columns

In [6]:
# 漫长的数据清洗过程
# 构建mapping，对有序变量'emp_length”、“grade”进行转换
mapping_dict = {
    'emp_length': {
        '10+ years': 10,
        '9 years': 9,
        '8 years': 8,
        '7 years': 7,
        '6 years': 6,
        '5 years': 5,
        '4 years': 4,
        '3 years': 3,
        '2 years': 2,
        '1 year': 1,
        '< 1 year': 0,
        'n/a': 0
    },
    'grade':{
        'A': 1,
        'B': 2,
        'C': 3,
        'D': 4,
        'E': 5,
        'F': 6,
        'G': 7
    }
}

data = data.replace(mapping_dict) #变量映射

In [7]:
# 去除无用的字符
data['int_rate'] = data.apply(lambda x: x['int_rate'].replace('%', ''), axis=1)
data['revol_util'] = data.apply(lambda x: str(x['revol_util']).replace('%', ''), axis=1)
data['term'] = data.apply(lambda x: str(x['term']).replace(' months', ''), axis=1)

In [8]:
# 筛选获得每列至少有2个分类特征的数组集
data = data.loc[:,data.apply(pd.Series.nunique)!=1]

In [9]:
# 空值选择填充
#使用None来填充缺失值
data['emp_length'].fillna(data['emp_length'].mode()[0]) # 工作年限使用众数填充

objectColumns = data.select_dtypes(include=['object']).columns # 筛选数据类型为object的数据
data[objectColumns] = data[objectColumns].fillna('None') #以分类“None”填充缺失值

#利用sklearn模块中的Imputer模块填充缺失值
numColumns = data.select_dtypes(include=[np.number]).columns
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)  # 针对axis=0 列来处理
imr = imr.fit(data[numColumns])
data[numColumns] = imr.transform(data[numColumns])

In [10]:
#使用Pandas replace函数定义新函数：
def coding(col, codeDict):
    colCoded = pd.Series(col, copy=True)
    for key, value in codeDict.items():
        colCoded.replace(key, value, inplace=True)
    return colCoded

#把贷款状态LoanStatus编码为违约=1, 正常=0:
pd.value_counts(data['loan_status'])
data['loan_status'] = coding(data['loan_status'], {'Current':0,'Fully Paid':0\
                                                     ,'In Grace Period':1\
                                                     ,'Late (31-120 days)':1\
                                                     ,'Late (16-30 days)':1\
                                                     ,'Charged Off':1\
                                                     ,"Issued":1\
                                                     ,"Default":1\
                                                    ,"Does not meet the credit policy. Status:Fully Paid":1\
                                         ,"Does not meet the credit policy. Status:Charged Off":1})
#统计"loan_status"数据的分布
data['loan_status'].value_counts()

0    342634
1     78460
Name: loan_status, dtype: int64

In [12]:
# 删除无效特征
data.select_dtypes(include=['object']).columns
data = data.drop(['emp_title', 'title', 'revol_util'], axis=1)

In [13]:
data = pd.get_dummies(data).reset_index(drop=True)

In [14]:
# 特征缩放
# 采用标准化的方法进行去量纲操作，加快算法收敛速度，采用scikit-learn模块preprocessing的子模块StandardScaler进行操作。
col = data.select_dtypes(include=['int64','float64']).columns
col = col.drop('loan_status') #剔除目标变量
dataStd = data # 复制数据至变量loans_ml_df

sc =StandardScaler() # 初始化缩放器
dataStd[col] =sc.fit_transform(dataStd[col]) #对数据进行标准化
dataStd.head() #查看经标准化后的数据

,loan_amnt,funded_amnt,funded_amnt_inv,installment,grade,emp_length,annual_inc,loan_status,dti,delinq_2yrs,...,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,last_credit_pull_d_Sep-2017,last_credit_pull_d_Sep-2018,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,1.103648,1.103648,1.104800,1.545612,0.170524,1.102994,-0.161806,0,-0.347593,0.703466,...,0,0,0,0,1,0,1,0,1,0
1,2.305330,2.305330,2.306957,1.584902,0.170524,1.102994,0.446714,0,-0.235044,-0.374474,...,0,0,0,0,1,0,1,0,1,0
2,-1.358049,-1.358049,-1.357871,-1.302085,0.170524,1.102994,-0.297033,0,-1.489958,-0.374474,...,0,0,0,0,1,0,1,0,1,0
3,-0.564706,-0.564706,-0.564214,-0.620516,2.471818,-0.839833,0.371433,0,0.700232,0.703466,...,0,0,0,0,1,0,1,0,1,0
4,0.555308,0.555308,0.556243,-0.037499,-0.596574,1.102994,-0.188851,0,-0.941848,-0.374474,...,0,0,0,0,0,1,1,0,1,0


In [15]:
#构建X特征变量和Y目标变量
y = dataStd['loan_status']
X = dataStd.drop(['loan_status'], axis=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
len(X) # 查看初始特征集合的数量

421094

In [16]:
names = dataStd.columns
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=10,random_state=123)#构建分类随机森林分类器
clf.fit(x_train, y_train) #对自变量和因变量进行拟合
names, clf.feature_importances_

(Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'grade',
        'emp_length', 'annual_inc', 'loan_status', 'dti', 'delinq_2yrs',
        ...
        'last_credit_pull_d_Sep-2015', 'last_credit_pull_d_Sep-2016',
        'last_credit_pull_d_Sep-2017', 'last_credit_pull_d_Sep-2018',
        'application_type_Individual', 'application_type_Joint App',
        'hardship_flag_N', 'hardship_flag_Y', 'debt_settlement_flag_N',
        'debt_settlement_flag_Y'],
       dtype='object', length=1997),
 array([0.03017882, 0.01798659, 0.01091435, ..., 0.00022015, 0.0179364 ,
        0.02368094]))

In [17]:
res = []
for feature in zip(clf.feature_importances_, names):
    res.append(feature)
res = sorted(res)
res = res[-13:]
res = [row[1] for row in res]

In [18]:
# 构建逻辑回归分类器
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression() 
clf1.fit(x_train[res], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
predicted1 = clf1.predict(x_test[res]) # 通过分类器产生预测结果
from sklearn.metrics import accuracy_score
print("Test set accuracy score: {:.5f}".format(accuracy_score(predicted1, y_test)))

Test set accuracy score: 0.98412


In [27]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))

gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)            

lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )

xgboost = XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)



In [ ]:
score = cv_rmse(ridge)
score = cv_rmse(lasso)
print("LASSO: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(elasticnet)
print("elastic net: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(svr)
print("SVR: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lightgbm)
print("lightgbm: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(gbr)
print("gbr: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(xgboost)
print("xgboost: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
print('START Fit')

print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(x_train[res]), np.array(y_train))

print('elasticnet')
elastic_model_full_data = elasticnet.fit(x_train[res], y_train)

print('Lasso')
lasso_model_full_data = lasso.fit(x_train[res], y_train)

print('Ridge')
ridge_model_full_data = ridge.fit(x_train[res], y_train)

print('Svr')
svr_model_full_data = svr.fit(x_train[res], y_train)

print('GradientBoosting')
gbr_model_full_data = gbr.fit(x_train[res], y_train)

print('xgboost')
xgb_model_full_data = xgboost.fit(x_train[res], y_train)

print('lightgbm')
lgb_model_full_data = lightgbm.fit(x_train[res], y_train)

In [ ]:
def blend_models_predict(X):
    return ((0.1 * elastic_model_full_data.predict(X)) + \
            (0.05 * lasso_model_full_data.predict(X)) + \
            (0.1 * ridge_model_full_data.predict(X)) + \
            (0.1 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.15 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.3 * stack_gen_model.predict(np.array(X))))

In [ ]:
print('RMSLE score on train data:')
print(rmsle(x_test, blend_models_predict(x_test)))